<a href="https://colab.research.google.com/github/5dongdong/PY_AI/blob/master/model/LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score

In [119]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [120]:
df = pd.read_csv('/content/drive/MyDrive/data/medical_noshow3.csv')  

FileNotFoundError: ignored

In [ ]:
df

In [ ]:
x = df[['Age',	'Scholarship',	'Hipertension',	'Diabetes',	'Alcoholism',	'SMS_received',	'Handicap_0',	'Handicap_1',	'Handicap_2',	'Handicap_3',	'Handicap_4',	'AwaitingTime',	'Num_App_Missed',	'Gender_F',	'Gender_M']]
# x = df[['Age',	'AwaitingTime',	'Num_App_Missed']]
y = df[['No-show']]   
# df = df.drop('Handicap_0', axis=1)
# df = df.drop('Handicap_2', axis=1)
# df = df.drop('Handicap_3', axis=1)
# df = df.drop('Handicap_4', axis=1)

x_train, x_test, y_train,y_test = train_test_split(
    x, y, train_size=0.8, shuffle=True, random_state=42
)
     

In [ ]:
df

In [ ]:
# 변수 설정
n_splits = 5    # 보통 홀수로 들어감
random_state = 42
scaler = MinMaxScaler()

In [ ]:
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, 
              random_state=random_state)


# Scaler
scaler.fit(x_train)                 
x_train = scaler.transform(x_train)   # train 은 fit, transform 모두 해줘야 함
x_test = scaler.transform(x_test) 

In [ ]:
param = {
    'num_leaves':[10, 20, 31],
    'max_depth':[5, 7, 8],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200],
    'feature_fraction':[0.5, 0.7, 0.8],
    'bagging_fraction':[0.5, 0.7, 0.8],
    'lambda_l1, lambda_l2':[1, 5, 10]
}
#bagging_fraction': (bagging_fraction=0.5, feature_fraction=0.7, lambda_l1, lambda_l2=10, learning_rate=0.2, max_depth=5, n_estimators=200, num_leaves=20)

In [ ]:
# 2. 모델
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

model = LGBMClassifier(num_leaves=49, max_depth=7,
                       learning_rate=0.07949705772341571, n_estimators=1751,
                       feature_fraction=0.022621757234206785, 
                       bagging_fraction=0.4089910928350561, 
                       lambda_l1=1.127987668851632,
                       lambda_l2=1.127987668851632)

# model = GridSearchCV(light, param, cv=kfold, verbose=1,
#                       refit=True, n_jobs=-1)

# 3.훈련
model.fit(x_train, y_train)

# print('최적의 파라미터 : ', model.best_params_)
# print('최적의 매개변수 : ', model.best_estimator_)
# print('best_score : ', model.best_score_)
# print('model score : ', model.score(x_test, y_test))



In [ ]:


#4. 평가, 예측
result = model.score(x_test,y_test)

score = cross_val_score( model, 
                        x_train, y_train,
                        cv = kfold )  #cv = cross validation_

y_predict = cross_val_predict(model,
                              x_test, y_test,
                              cv = kfold)

acc = accuracy_score(y_test, y_predict)

print('결과 acc : ', result)
print('cv pred acc : ', acc )


# LGBMClassifier(bagging_fraction=0.5, feature_fraction=0.7, lambda_l1=10,
#                lambda_l2=10, learning_rate=0.2, max_depth=5, n_estimators=200,
#                num_leaves=20)
# 결과 acc :  0.9204216431415129
# cv pred acc :  0.9182500904813609

# LGBMClassifier(feature_fraction=0.6, max_depth=10, min_data_in_leaf=50,
#                n_estimators=200, num_leaves=32, reg_alpha=0.1, reg_lambda=0.01)
# 결과 acc :  0.9207383279044516
# cv pred acc :  0.9152642055736518

# LGBMClassifier(bagging_fraction=0.5, feature_fraction=0.5, lambda_l1=10,
#                lambda_l2=10, max_depth=8)
# 결과 acc :  0.9201501990589939
# cv pred acc :  0.9188382193268186


# LGBMClassifier(bagging_fraction=0.4089910928350561,
#                feature_fraction=0.022621757234206785,
#                lambda_l1=1.127987668851632, lambda_l2=1.127987668851632,
#                learning_rate=0.07949705772341571, max_depth=7,
#                n_estimators=1751, num_leaves=49)
# 결과 acc :  0.9192453854505972
# cv pred acc :  0.9181143684401013

In [ ]:
from sklearn.feature_selection import SelectFromModel

thresholds = model.feature_importances_

for thresh in thresholds:
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_x_train = selection.transform(x_train)
    select_x_test = selection.transform(x_test)
    print(select_x_train.shape, select_x_test.shape)

    selection_model = LGBMClassifier(feature_fraction=0.6, max_depth=10, min_data_in_leaf=50, n_estimators=200, num_leaves=32, reg_alpha=0.1, reg_lambda=0.01)
    selection_model.fit(select_x_train, y_train)
    y_predict = selection_model.predict(select_x_test)
    score = accuracy_score(y_test, y_predict)
    print("Thresh=%.3f, n=%d, acc:%.2f%%"
        %(thresh, select_x_train.shape[1], score*100)) 

In [ ]:
import matplotlib.pyplot as plt
feature_importances = model.feature_importances_
fig, ax = plt.subplots()
ax.barh(x.columns, feature_importances)
ax.set_xlabel('Feature Importance')
ax.set_ylabel('Features')
ax.set_title('Feature Importance Plot')
plt.show()